In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../data/retail_transactional_dataset.csv")

In [4]:
# Backup raw data (optional)
df_raw = df.copy()

In [5]:
pii_columns = ['Name', 'Email', 'Phone', 'Address', 'Zipcode']
df = df.drop(columns=pii_columns)

In [6]:
df.isnull().sum().sort_values(ascending=False)

Total_Purchases     361
Date                359
Amount              357
Year                350
Total_Amount        350
Time                350
Shipping_Method     337
Transaction_ID      333
Gender              317
Customer_ID         308
Payment_Method      297
Income              290
Product_Category    283
State               281
Product_Brand       281
Month               273
Country             271
City                248
Order_Status        235
Customer_Segment    215
Ratings             184
Feedback            184
Age                 173
Product_Type          0
products              0
dtype: int64

In [7]:
# Drop rows that cannot be used
df = df.dropna(subset=['Transaction_ID', 'Customer_ID', 'Date', 'Amount', 'Total_Purchases'])

# Fill missing categorical data
df['Shipping_Method'] = df['Shipping_Method'].fillna('Standard')
df['Payment_Method'] = df['Payment_Method'].fillna('Unknown')
df['Order_Status'] = df['Order_Status'].fillna('Processing')
df['Gender'] = df['Gender'].fillna('Unknown')
df['Income'] = df['Income'].fillna('Unknown')
df['Customer_Segment'] = df['Customer_Segment'].fillna('Regular')
df['Product_Category'] = df['Product_Category'].fillna('Misc')
df['Product_Brand'] = df['Product_Brand'].fillna('Generic')
df['City'] = df['City'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')
df['Country'] = df['Country'].fillna('Unknown')
df['Feedback'] = df['Feedback'].fillna('No Feedback')
df['Ratings'] = df['Ratings'].fillna(0)

# Fill missing numeric data with median if needed
df['Age'] = df['Age'].fillna(df['Age'].median())

In [8]:
df.isnull().sum().sort_values(ascending=False)

Time                349
Year                348
Total_Amount        347
Month               266
Country               0
Customer_ID           0
City                  0
State                 0
Transaction_ID        0
Customer_Segment      0
Income                0
Gender                0
Age                   0
Date                  0
Total_Purchases       0
Amount                0
Product_Category      0
Product_Brand         0
Product_Type          0
Feedback              0
Shipping_Method       0
Payment_Method        0
Order_Status          0
Ratings               0
products              0
dtype: int64

In [9]:
# Convert Time to datetime first
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')

# Fill missing time with median time
median_time = df['Time'].median()
df['Time'] = df['Time'].fillna(median_time)

In [10]:
df.isnull().sum().sort_values(ascending=False)

Year                348
Total_Amount        347
Month               266
Customer_ID           0
Country               0
Age                   0
City                  0
State                 0
Transaction_ID        0
Customer_Segment      0
Income                0
Gender                0
Date                  0
Time                  0
Total_Purchases       0
Amount                0
Product_Category      0
Product_Brand         0
Product_Type          0
Feedback              0
Shipping_Method       0
Payment_Method        0
Order_Status          0
Ratings               0
products              0
dtype: int64

In [11]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

AttributeError: Can only use .dt accessor with datetimelike values

In [12]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [13]:
df['Date'].dtype

dtype('<M8[us]')

In [14]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [15]:
df[['Date', 'Year', 'Month']].head()

,Date,Year,Month
0,2023-09-18,2023.0,9.0
1,2023-12-31,2023.0,12.0
2,2023-04-26,2023.0,4.0
3,NaT,NaN,NaN
4,NaT,NaN,NaN


In [16]:
df = df.dropna(subset=['Date'])

In [17]:
df[['Date', 'Year', 'Month']].head(10)

,Date,Year,Month
0,2023-09-18,2023.0,9.0
1,2023-12-31,2023.0,12.0
2,2023-04-26,2023.0,4.0
5,2023-09-21,2023.0,9.0
6,2023-06-26,2023.0,6.0
7,2023-03-24,2023.0,3.0
10,2023-07-20,2023.0,7.0
11,2023-06-21,2023.0,6.0
14,2023-11-18,2023.0,11.0
15,2023-06-15,2023.0,6.0


In [18]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [19]:
print("Number of rows after dropping invalid dates:", df.shape[0])

Number of rows after dropping invalid dates: 182064


In [20]:
df['Order_YearMonth'] = df['Date'].dt.to_period('M')
df[['Date', 'Order_YearMonth']].head()

,Date,Order_YearMonth
0,2023-09-18,2023-09
1,2023-12-31,2023-12
2,2023-04-26,2023-04
5,2023-09-21,2023-09
6,2023-06-26,2023-06


In [21]:
df['Order_Hour'] = df['Time'].dt.hour
df[['Time', 'Order_Hour']].head()

,Time,Order_Hour
0,1900-01-01 22:03:55,22
1,1900-01-01 08:42:04,8
2,1900-01-01 04:06:29,4
5,1900-01-01 23:24:27,23
6,1900-01-01 13:35:51,13


In [22]:
df['Revenue'] = df['Amount'] * df['Total_Purchases']
df[['Amount', 'Total_Purchases', 'Revenue']].head()

,Amount,Total_Purchases,Revenue
0,108.028757,3.0,324.086270
1,403.353907,2.0,806.707815
2,354.477600,3.0,1063.432799
5,296.291806,4.0,1185.167224
6,315.057648,2.0,630.115295


In [24]:
latest_date = df['Date'].max()
df['Recency_Days'] = (latest_date - df['Date']).dt.days
df[['Date', 'Recency_Days']].head()

,Date,Recency_Days
0,2023-09-18,164
1,2023-12-31,60
2,2023-04-26,309
5,2023-09-21,161
6,2023-06-26,248


In [25]:
first_purchase = df.groupby('Customer_ID')['Date'].min()
df = df.merge(first_purchase.rename('First_Purchase_Date'), on='Customer_ID')
df['Customer_Tenure'] = (df['Date'] - df['First_Purchase_Date']).dt.days
df[['Customer_ID', 'Date', 'First_Purchase_Date', 'Customer_Tenure']].head()

,Customer_ID,Date,First_Purchase_Date,Customer_Tenure
0,37249.0,2023-09-18,2023-06-14,96
1,69749.0,2023-12-31,2023-12-31,0
2,30192.0,2023-04-26,2023-04-26,0
3,41289.0,2023-09-21,2023-08-31,21
4,97285.0,2023-06-26,2023-06-26,0


In [26]:
df.info()
df.describe()
df.head()

<class 'pandas.DataFrame'>
RangeIndex: 182064 entries, 0 to 182063
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Transaction_ID       182064 non-null  float64       
 1   Customer_ID          182064 non-null  float64       
 2   City                 182064 non-null  str           
 3   State                182064 non-null  str           
 4   Country              182064 non-null  str           
 5   Age                  182064 non-null  float64       
 6   Gender               182064 non-null  str           
 7   Income               182064 non-null  str           
 8   Customer_Segment     182064 non-null  str           
 9   Date                 182064 non-null  datetime64[us]
 10  Year                 182064 non-null  int32         
 11  Month                182064 non-null  int32         
 12  Time                 182064 non-null  datetime64[us]
 13  Total_Purchases      1820

,Transaction_ID,Customer_ID,City,State,Country,Age,Gender,Income,Customer_Segment,Date,...,Payment_Method,Order_Status,Ratings,products,Order_YearMonth,Order_Hour,Revenue,Recency_Days,First_Purchase_Date,Customer_Tenure
0,8691788.0,37249.0,Dortmund,Berlin,Germany,21.0,Male,Low,Regular,2023-09-18,...,Debit Card,Shipped,5.0,Cycling shorts,2023-09,22,324.086270,164,2023-06-14,96
1,2174773.0,69749.0,Nottingham,England,UK,19.0,Female,Low,Premium,2023-12-31,...,Credit Card,Processing,4.0,Lenovo Tab,2023-12,8,806.707815,60,2023-12-31,0
2,6679610.0,30192.0,Geelong,New South Wales,Australia,48.0,Male,Low,Regular,2023-04-26,...,Credit Card,Processing,2.0,Sports equipment,2023-04,4,1063.432799,309,2023-04-26,0
3,6095326.0,41289.0,Brisbane,New South Wales,Australia,58.0,Female,Medium,Premium,2023-09-21,...,PayPal,Pending,4.0,Lenovo Tab,2023-09,23,1185.167224,161,2023-08-31,21
4,5434096.0,97285.0,Kitchener,Ontario,Canada,29.0,Female,Low,New,2023-06-26,...,Cash,Processing,1.0,QLED TV,2023-06,13,630.115295,248,2023-06-26,0


In [27]:
df.to_csv('../data/cleaned_retail_data.csv', index=False)